In [1]:
#!pip install --upgrade protobuf
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import LlamaCpp, HuggingFacePipeline # On va utiliser Vigogne pour les réponses en FR
                                    # Et ce modèle est basé sur LlamaCpp. On utilise HuggingFacePipeline pour
                                    # les résumés des infos wikipedia
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM


2023-04-21 09:54:38.918435: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-21 09:54:39.510743: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
Question="À quelle date TCP/IP a été créé ?"

In [3]:
from langchain.utilities import WikipediaAPIWrapper
wikipedia = WikipediaAPIWrapper()
wikipedia.wiki_client.set_lang('fr')

In [10]:
wiki_summ=wikipedia.wiki_client.page(wikipedia.wiki_client.search(Question)[0]).summary
#print(wiki_summ)

In [12]:
##print(wikipedia.run('kdjhekhdc'))

In [13]:

#if wiki_summ == "No good Wikipedia Search Result was found":
#    wiki_summ = ""
#    template = """{wiki_summ} {result}
#    """
#    n_ctx=1000
#    max_tokens=800
#    llm = LlamaCpp(model_path="../../twitch/vigogne/llama.cpp/models/7B/ggml-model-q4_0.bin",temperature=0.1,n_ctx=n_ctx,max_tokens=max_tokens) #,max_tokens=8100,n_ctx=8192) 
#    prompt = PromptTemplate(template=template, input_variables=["wiki_summ","result"])

#else:
#    template = """
#    Tu es un assistant expert pour répondre aux questions des gens. Sachant que {wiki_summ} et en te basant sur tes propres données,\n
#    Que peux-tu me dire sur {result} 
#    """
#    #wiki_summ=wikipedia.wiki_client.page(Question).summary
#    n_ctx=20000
#    max_tokens=10000
#    prompt = PromptTemplate(template=template, input_variables=["wiki_summ","result"])
#    print(prompt)
#    llm = LlamaCpp(model_path="../../twitch/vigogne/llama.cpp/models/7B/ggml-model-q4_0.bin",temperature=0.1,n_ctx=n_ctx,max_tokens=max_tokens) #,max_tokens=8100,n_ctx=8192) 




In [14]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

#csebuetnlp/mT5_multilingual_XLSum ??
model_id = 'bofenghuang/flan-t5-large-french-dialogue-summarization'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=True,device_map='sequential')

pipe = pipeline(
    "text2text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=1000,
    min_length=50
)

local_llm = HuggingFacePipeline(pipeline=pipe)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


In [15]:
print(local_llm(wiki_summ))

TCP est un protocole de transport fiable, en mode connecté, documenté dans la RFC 9293 de l'IETF. TCP est situé au-dessus de IP et en correspondance à la couche transport, intermédiaire de la couche réseau et de la couche session.


In [16]:
print(wiki_summ)

Transmission Control Protocol (littéralement, « protocole de contrôle de transmissions »), abrégé TCP, est un protocole de transport fiable, en mode connecté, documenté dans la RFC 9293 de l’IETF.
Dans le modèle Internet, aussi appelé modèle TCP/IP, TCP est situé au-dessus de IP. Dans le modèle OSI, il correspond à la couche transport, intermédiaire de la couche réseau et de la couche session. Les applications transmettent des flux de données sur une connexion réseau. TCP découpe le flux d’octets en segments dont la taille dépend de la MTU du réseau sous-jacent (couche liaison de données).
TCP a été développé en 1973 puis adopté pour Arpanet en 1983, remplaçant NCP (RFC 801).


